In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

# 데이터 불러오기

In [ ]:
df = pd.read_excel('data.xlsx')
display(df)

# 라인 밸런싱 알고리즘 작성
라인밸런싱 알고리즘에 필요한 함수들을 먼저 작성한다

### 선후 관계를 체크하는 `get_relation_boolean_mask` 함수 작성
선후 관계를 만족하는 데이터들만 추출하는 함수를 작성한다. 이 때 선후 관계가 NaN인 것들은 무조건 True를 반환하도록 한다.

In [ ]:
def get_relation_boolean_mask(successful_candidates, cumulative_time):
    # 제일 먼저 선후 관계를 만족한 데이터들을 추출합니다
    # 각 행을 조사해서 선후관계표에 포함된 번호들이 모두 succsesful_candidate에 포함되었는지 확인합니다
    # 만약 그렇다면 True를, 아니면 False를 리턴한다
    candidates_satisfy_relation = df.iloc[3:].apply(lambda row: True if 
        all([ int(num) in successful_candidates for num in str(row['선후관계표']).split(',') ]) 
        else False, axis=1 )
    # 선후관계가 존재하지 않는 작업은 무조건 True이므로 3개의 True 값을 포함하는 Series를 결합해줍니다
    candidates_satisfy_relation = pd.concat([pd.Series([True, True, True]), candidates_satisfy_relation], axis=0)
    
    return candidates_satisfy_relation

### 조건에 만족하는 candidates만 추출하는 `make_candidates` 함수 작성
위 에서 체크한 선후 관계와 함께 이미 수행된 작업은 아닌지, 누적 작업 시간이 CT를 초과하진 않는지를 체크하는 함수를 작성한다

In [ ]:
def make_candidates(successful_candidates, cumulative_time):
    candidates = df[   
        # 이미 수행된 작업이 아니고(작업 이름이 successful_candidates에 존재하지 않고)
        [ name not in successful_candidates for name in df['작업이름'] ]
        # 누적 작업 시간이 CT를 넘지 않으며
        & (cumulative_time + df['요소작업시간'].values <= CT_a)
        # 선후관계를 만족하는 데이터들을 모두 가져옵니다
        & get_relation_boolean_mask(successful_candidates, cumulative_time)
    ]
    
    return candidates

### 조건을 만족하는 candidates로부터 랜덤하게 candidate를 선정하는 `select_candidate` 함수 작성
조건을 만족하는 candidates들로 부터 특정 candidate를 선정하기 위해 균일한 분포에서 난수를 하나 생성하고, 그 난수를 index로 사용한다

In [ ]:
def select_candidate(candidates, stations):
    global total_candidates, successful_candidates, cumulative_time
    # 이 난수는 주어진 값을 동일한 분포로 랜덤하게 생성합니다
    # 생성된 난수는 선택할 작업의 이름을 나타냅니다
    r = np.random.randint(1, len(candidates)+1, 1)[0]
    
    successful_candidates.append(candidates.iloc[r-1]['작업이름'])
    cumulative_time += candidates.iloc[r-1]['요소작업시간']
    
    total_candidates.append(candidates.iloc[r-1]['작업이름'])
    stations.append(candidates.iloc[r-1]['작업이름'])

# 라인 밸런싱 수행
총 5번의 라인밸런싱 알고리즘(COMSOAL)을 수행한다

In [ ]:
print('-'*20 + ' COMSOAL LINE BALANCING ' + '-'*20)
# COMSOAL 알고리즘을 5번 반복합니다
for test in range(5):
    # 현재까지 완료된 작업들을 저장하는 빈 배열을 만듭니다
    successful_candidates = []
    # 완료된 작업들을 모두 저장하는 배열을 만듭니다
    total_candidates = []
    # 실제 CT는 0으로 초기화합니다
    CT_a = 0
    print('-'*64)
    print(f'test#{test}')
    # 모든 작업장에 작업이 배치될 때 까지 반복한다
    # 즉, total_candidates의 길이와 df의 길이가 동일해질 때 까지 반복합니다
    i = 1
    while len(total_candidates) < len(df):
        # 작업 시간 누산기를 생성합니다. 작업장 별 누적 작업 시간을 저장하고 CT와 비교합니다
        cumulative_time = 0
        # 조건을 만족하는 작업 목록을 추출하여 candidates에 저장합니다
        candidates = make_candidates(successful_candidates, cumulative_time)
        # 해당 작업장 별 작업을 저장하는 배열을 생성합니다
        stations = []
        # 조건을 만족하는 작업들 중에서 랜덤하게 작업을 선택해나가고,
        # 더이상 조건을 만족하는 작업이 존재하지 않을 때 까지 반복합니다
        while len(candidates) > 0:
            # 난수를 생성하여 candidates들 중에서 하나의 작업을 랜덤하게 선정합니다
            select_candidate(candidates, stations)
            # 선택된 작업을 완료하면, 선행 관계가 새롭게 구성되므로
            # 조건을 만족하는 작업들을 새롭게 candidates에 저장합니다
            candidates = make_candidates(successful_candidates, cumulative_time)
        CT_a = max(CT_a, cumulative_time)
        print(f'station {i} = {stations}, time = {cumulative_time}')
        i += 1
    print(f'편성 효율: {150/((i-1) * CT_a)}')

# Note

- 대당 총 작업 시간 $T_wc = 2.5 hr$
- 시간 당 생산 수요량 $R_p = \frac{320}{160 hr} = 2 units/hr$
- CT $T_c = 60 / 2 = 30$
- 최소 작업장 수 $w* = \frac{150 min}{30} = 5$

## 알고리즘

1. 작업장 리스트를 생성한다. 즉 빈 배열을 만든다.
2. 리스트에서 후보 작업들을 선정한다. 이 때 선후 관계와 CT를 고려해야한다.
   1. 만약 후보 작업들을 선택했을 때 CT를 초과하게 된다면 후보에서 제외한다.
   2. 만약 선후 관계가 만족되지 않은 경우 후보에서 제외한다.
   3. 후보 작업이 존재하지 않으면 종료한다.
3. 후보의 개수가 N개일 때, 난수를 생성하고 이를 $r$ 이라고 하면 $r \le \frac {1}{N}$ 이면 1번째, $r \le \frac {2}{N}$ 이면 2번째, ..., $r \le \frac {N}{N}$ 이면 N번째 후보를 선택한다($0 \le r \le 1$).
4. 선택된 후보를 작업장 리스트에 추가한다. 더 이상 후보 작업들이 존재하지 않을 때 까지 2~4 단계를 반복한 뒤 다시 1단계로 돌아간다.
5. 더이상 작업장 리스트를 만들 수 없다면 라인 편성 효율을 계산한다. 즉 총 작업 시간 $T_c$를 작업장 수 N과 CT의 곱으로 나눈 값 $$\frac {T_c}{N * CT}$$ 를 계산한다.

## 지희야 숙제 하나 내줄게
지희야 랜덤으로 candidate를 선정하는 `select_candidate` 함수 있지? 여기서 랜덤 난수 r을 그냥 정수로 선택해서 index로 활용하고 있는데 이 부분을 너가 수정해봐! 즉 과제에 맞게 제출하려면 확률(0 ~ 1 사이 값)로 계산하고, 그에 맞게 candidate를 선택할 수 있도록!! 직접 해보구 잘 안되면 또 연락해줘 :)